Deploying a PMML model in a Java-based system is a common use case, especially in enterprise environments. Since PMML (Predictive Model Markup Language) is a standardized format, you can use Java libraries to load and execute the model. Below is a step-by-step guide to deploy your PMML model in a Java-based system:
Step 1: Choose a Java PMML Library

There are several Java libraries for working with PMML models. The most popular one is JPMML (Java PMML Evaluator). It provides a robust and efficient way to load and evaluate PMML models.

    JPMML Website: https://github.com/jpmml/jpmml-evaluator

    Maven Dependency:
`
    <dependency>
        <groupId>org.jpmml</groupId>
        <artifactId>pmml-evaluator</artifactId>
        <version>1.6.3</version>
    </dependency>
    <dependency>
        <groupId>org.jpmml</groupId>
        <artifactId>pmml-evaluator-extension</artifactId>
        <version>1.6.3</version>
    </dependency>
`

Step 2: Write Java Code to Load and Evaluate the PMML Model

In [ ]:

import org.dmg.pmml.PMML;
import org.jpmml.evaluator.*;
import org.jpmml.model.PMMLUtil;

import java.io.File;
import java.util.HashMap;
import java.util.LinkedHashMap;
import java.util.List;
import java.util.Map;

public class PMMLPredictor {

    public static void main(String[] args) throws Exception {
        // Load the PMML model
        File pmmlFile = new File("bestmodel/xgboost_model.pmml");
        PMML pmml = PMMLUtil.unmarshal(pmmlFile);

        // Create a model evaluator
        ModelEvaluatorFactory modelEvaluatorFactory = ModelEvaluatorFactory.newInstance();
        ModelEvaluator<?> modelEvaluator = modelEvaluatorFactory.newModelEvaluator(pmml);

        // Prepare input data
        Map<String, Object> inputData = new HashMap<>();
        inputData.put("sex", 0);
        inputData.put("age_group", 3);
        inputData.put("Initiation_Year", 1);
        inputData.put("entrypoint", 5);
        inputData.put("regime_change", 1);
        inputData.put("pregnancy", 0);
        inputData.put("TB", 1);
        inputData.put("facility_level", 3);
        inputData.put("Contact", 0);
        inputData.put("season", 1);
        inputData.put("who_stage", 1);
        inputData.put("treatment_duration", 4);
        inputData.put("advanced_hiv", 0);
        inputData.put("no_IIT_ever", 1);
        inputData.put("no_IIT_in_last12months", 0);
        inputData.put("visits_current_regimen", 0);
        inputData.put("number_visits", 0);

        // Evaluate the model
        Map<String, ?> results = evaluateModel(modelEvaluator, inputData);

        // Print the prediction
        System.out.println("Prediction: " + results);
    }

    private static Map<String, ?> evaluateModel(ModelEvaluator<?> modelEvaluator, Map<String, Object> inputData) {
        // Prepare input fields
        List<InputField> inputFields = modelEvaluator.getInputFields();

        // Map input data to the model's input fields
        Map<FieldName, FieldValue> arguments = new LinkedHashMap<>();
        for (InputField inputField : inputFields) {
            FieldName fieldName = inputField.getName();
            Object rawValue = inputData.get(fieldName.getValue());
            FieldValue fieldValue = inputField.prepare(rawValue);
            arguments.put(fieldName, fieldValue);
        }

        // Evaluate the model
        Map<FieldName, ?> results = modelEvaluator.evaluate(arguments);

        // Prepare the output
        List<TargetField> targetFields = modelEvaluator.getTargetFields();
        Map<String, Object> output = new LinkedHashMap<>();
        for (TargetField targetField : targetFields) {
            FieldName fieldName = targetField.getName();
            Object value = results.get(fieldName);
            output.put(fieldName.getValue(), value);
        }

        return output;
    }
}


Step 3: Compile and Run the Java Program
    Save the above code in a file named PMMLPredictor.java.

    Compile the Java program:

    javac -cp "path/to/jpmml-evaluator-1.6.3.jar:path/to/pmml-model-1.6.3.jar" PMMLPredictor.java

    Run the program:
    java -cp ".:path/to/jpmml-evaluator-1.6.3.jar:path/to/pmml-model-1.6.3.jar" PMMLPredictor

Step 4: Integrate into Your Java System

    If you’re deploying this in a web application (e.g., Spring Boot), you can wrap the model evaluation logic in a service class.

    @Service
    public class PredictionService {

        private ModelEvaluator<?> modelEvaluator;

        @PostConstruct
        public void initialize() throws Exception {
            File pmmlFile = new File("bestmodel/xgboost_model.pmml");
            PMML pmml = PMMLUtil.unmarshal(pmmlFile);
            ModelEvaluatorFactory modelEvaluatorFactory = ModelEvaluatorFactory.newInstance();
            this.modelEvaluator = modelEvaluatorFactory.newModelEvaluator(pmml);
        }

        public Map<String, ?> predict(Map<String, Object> inputData) {
            return evaluateModel(modelEvaluator, inputData);
        }

        private Map<String, ?> evaluateModel(ModelEvaluator<?> modelEvaluator, Map<String, Object> inputData) {
            // Same evaluation logic as above
        }
    }